---
已知研究对象分成若干类型（或组别）并已取得各种类型的一批已知样品的观测数据，然后在此基础上根据某些准则建立判别式，并对未知类型的样品进行判别分类

### 11.2.1 距离判别法
分别计算各类的重心，对任意给定的一个样品，若它与第 $i$ 类的重心距离最近，就认为它来自第 $i$ 类
> 因此，距离判别法又称为最近邻方法

两个样本的情形：
若协方差阵 $\boldsymbol \Sigma^{(1)}$ 与 $\boldsymbol \Sigma^{(2)}$ 不相等，则判别函数为
$$
\begin{align*}
W(\boldsymbol X)&=d^2(\boldsymbol X, G_1) - d^2(\boldsymbol X, G_2)\\
\end{align*}
$$
其中距离为马氏距离[^1]。判别规则为
$$
\begin{cases}
\boldsymbol X\in G_1,\quad &{\text 若} W(\boldsymbol X)<0\\
\boldsymbol X\in G_2,\quad &{\text 若} W(\boldsymbol X)>0\\
{\text 待判},\quad &{\text 若} W(\boldsymbol X)=0 
\end{cases}
$$
多个样本的情形：
若协方差阵 $\boldsymbol \Sigma^{(1)}$ 与 $\boldsymbol \Sigma^{(2)}$ 不相等，则判别函数为
$$
\begin{align*}
V_{ij}(\boldsymbol X)&=d^2(\boldsymbol X, G_i) - d^2(\boldsymbol X, G_j),\ i,j=1,2,\cdots,k\\
\end{align*}
$$
其中距离为马氏距离[^1]。判别规则为
$$
\begin{cases}
\boldsymbol X\in G_i,\quad &{\text 若} V_{ij}(\boldsymbol X)<0,\forall j\neq i\\
{\text 待判},\quad &{\text 若} W(\boldsymbol X)=0 
\end{cases}
$$
> 1: 亦可为别的距离，如欧式距离
#### 1.

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
sp.init_printing(use_latex=True)
from numpy.linalg import inv
from scipy.spatial.distance import mahalanobis

In [2]:
data = pd.read_csv('../../11第11章  聚类分析与判别分析模型/data11_9.txt', sep='\t', header=None).values
af_a, af_l = data[0], data[1]
apf_a, apf_l = data[2, :-3], data[3, :-3]
G1 = np.vstack([af_a, af_l])
G2 = np.vstack([apf_a, apf_l])
mu1 = G1.mean(axis=1)
mu2 = G2.mean(axis=1)
cov1 = np.cov(G1, ddof=1)
cov2 = np.cov(G2, ddof=1)
W = []

x1, x2 = sp.symbols('x1, x2', nonneg=True)
X = sp.Matrix([x1, x2])
Mu1 = sp.Matrix([mu1]).T
Mu2 = sp.Matrix([mu2]).T
Cov1 = sp.Matrix(cov1)
Cov2 = sp.Matrix(cov2)
d1s = (X - Mu1).T @ Cov1.inv() @ (X - Mu1)
d2s = (X - Mu2).T @ Cov2.inv() @ (X - Mu2)
W = (d1s - d2s).expand()
W_spf = sp.lambdify('x1, x2', W, 'numpy')
Wf = lambda x: W_spf(x[0], x[1])[0,0]

sample = np.array([[1.24, 1.80], [1.28, 1.84], [1.40, 2.04]])
W_sample = []
for s in sample:
    W_sample.append(Wf(s))
W_sample

In [3]:
W_known_sample = []
known_sample = np.vstack([np.hstack([af_a, apf_a]), np.hstack([af_l, apf_l])]).T
for s in known_sample:
    W_known_sample.append(Wf(s))
W_known_sample

对于已知的 $15$ 个样本点，上述判别模型的误判率为 $0$
#### 2.

In [4]:
df = pd.read_excel('../../11第11章  聚类分析与判别分析模型/data11_10.xlsx', header=None)
df1 = df[df[5]==1]
df2 = df[df[5]==2]
G1 = df1.values[:, :-1].T.astype(float)     # 变量为行，样本为列
G2 = df2.values[:, :-1].T.astype(float)
Mu1 = sp.Matrix(G1.mean(axis=1))
Mu2 = sp.Matrix(G2.mean(axis=1))
Cov1 = sp.Matrix(np.cov(G1, ddof=1))
Cov2 = sp.Matrix(np.cov(G2, ddof=1))

x1, x2, x3, x4, x5 = sp.symbols('x1, x2, x3, x4, x5', nonneg=True)
X = sp.Matrix([x1, x2, x3, x4, x5])
d1s = (X - Mu1).T @ Cov1.inv() @ (X - Mu1)
d2s = (X - Mu2).T @ Cov2.inv() @ (X - Mu2)
W = (d1s - d2s).expand()
W_spf = sp.lambdify('x1, x2, x3, x4, x5', W, 'numpy')
Wf = lambda x: W_spf(x[0], x[1], x[2], x[3], x[4])[0,0]
Classify_f = lambda x: 1 if Wf(x) < 0 else 2

sample = df[df[5]=="待判"].values[:, :-1].astype(float) # 样本为行，变量为列
W_sample = []
classes = []
for s in sample:
    W_sample.append(Wf(s))
    classes.append(Classify_f(s))
W_sample, classes

In [5]:
classes_known = []
known_sample = np.vstack([G1.T, G2.T])
for s in known_sample:
    classes_known.append(Classify_f(s))
classes_known

对于已知的 $16$ 个样本点，上述判别模型的误判率为 $0$

### 11.2.2 Fisher 判别
投影的思想

判别函数
$$
\begin{align*}
W(\boldsymbol X)=\left(\boldsymbol X-\dfrac{1}{2}\left(\boldsymbol \mu_1+\boldsymbol\mu_2\right)\right)^{\rm T}\boldsymbol\Sigma^{-1}\left(\boldsymbol\mu_1-\boldsymbol\mu_2\right)
\end{align*}
$$
其中 $\boldsymbol\Sigma$ 为两类共同的协方差矩阵，判别规则
$$
\begin{cases}
\boldsymbol X\in G_1,\quad &\boldsymbol X\geq0\\
\boldsymbol X\in G_2,\quad &\boldsymbol X<0
\end{cases}
$$

In [6]:
data = np.loadtxt('../../11第11章  聚类分析与判别分析模型/data11_11.txt')
G1 = data[:5].T
G2 = data[5:10].T
sample = data[10:]
Mu1 = sp.Matrix(G1.mean(axis=1))
Mu2 = sp.Matrix(G2.mean(axis=1))
Cov = sp.Matrix(np.cov(np.hstack([G1, G2]), ddof=1))

x1, x2, x3, x4 = sp.symbols('x1, x2, x3, x4', real=True)
X = sp.Matrix([x1, x2, x3, x4])
W = (X - (Mu1 + Mu2)/2).T @ Cov.inv() @ (Mu1 - Mu2)
Wf_raw = sp.lambdify('x1, x2, x3, x4', W, 'numpy')
Wf = lambda x: Wf_raw(x[0], x[1], x[2], x[3])
Classify_f = lambda x: 'A' if Wf(x) >= 0 else 'B'

classes_pred = []
for s in sample:
    classes_pred.append(Classify_f(s))
classes_pred

['B', 'A', 'A', 'B', 'B', 'A']

In [7]:
classes_known = []
known_sample = np.vstack([G1.T, G2.T])
for s in known_sample:
    classes_known.append(Classify_f(s))
classes_known

['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B']

误判率 $0\%$

### 11.2.3 判别标准的评价
设属于 $G_1$ 而被误判为属于 $G_2$ 的个数为 $N_1$，属于 $G_2$ 而被误判为属于 $G_1$ 的个数为 $N_2$，两类总体的样本总数为 $N$，则误判率的估计为 $$\hat P=\dfrac{N_1+N_2}{N}$$
#### 1. 回代误判率
前面所使用的误判率的估计方法的就是回代法，直接回代判断，易于计算。但是 $\hat P$ 是由建立判别函数的数据回代判别函数而得到的，因此其往往比真实误判率小
#### 2. 交叉误判率
每次删除一个样品，利用其余的样品建立判别函数对被删样品进行判别。对训练样品中的所有样品都做如上分析，以其误判的比例作为误判率
> - 用交叉法估计误判率是较为合理的
> 
> - 当训练的样品足够多时，可以留出一些已知类别的样品不参加建立判别函数而是作为检验集，并把误判的比率作为误判率的估计

sklearn 库中 `sklearn.model_selection` 模块的 `cross_val_score` 函数可以计算交叉检验的精度
```python
cross_val_score(model, x0, y0, cv=k)
```
- `model`：所建立的模型
- `x0`：已知样本点的数据
- `y0`：样本的标号值
- `cv`：把已知样本点分为 `k` 组，其中 `k-1` 组被训练用作数据集，剩下一组被用作检验集。这样一共可以对分类器 (model) 进行 $k$ 次训练，得到 $k$ 个训练结果
- `RETURN`：返回值为每组评估数据分类的准确率

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score

data = pd.read_excel('../../11第11章  聚类分析与判别分析模型/data11_10.xlsx', header=None).values
x0 = data[:-2, :-1].astype(float)
y0 = data[:-2, -1].astype(float)
md = LDA()
print(cross_val_score(md, x0, y0, cv=2))

[0.875 0.75 ]
